In [1]:
# HIDE CODE
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
matplotlib.rcParams.update({'font.size': 12})


import warnings
import sys
import os
sys.path.append('/home/server/gli-data-science/')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

import os
import ds_db
import helper_db
from helper import transform_to_rupiah, rupiah_format

import pickle
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display, HTML, display_html, IFrame
import ipywidgets as ipyw

def side_by_side_display(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    styles = [
        dict(selector="caption", props=[("caption-side", "center"), ("font-size", "100%"), ("color", )])]
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline; font-size:85%' ").set_precision(2).set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0\xa0"

    display(HTML(output))

    
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, PoissonRegressor, Ridge, Lasso, RANSACRegressor, RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, export_text, DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVR, SVC, LinearSVC
import xgboost as xgb 


from joblib import dump, load
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split


import textwrap
def split_label(list_label):
    list_label = list(list_label)
    list_label = ["<br>".join(textwrap.wrap(t, width=12)) for t in list_label ]
    return list_label

### ide kasar
> 
* daily job to loop all member alfagift 10 juta
* compute feature trx voucher used, # of trx, # of sales
* compute feature event app site open / view product / atc
* updating each feature and re predict churn feature
* lets play in month - 3 for each define trx and event feature

In [2]:
df_pv_ = pd.read_csv('./feature/pv_30Jun22.csv')
df_pv_ = df_pv_.rename(columns = {'TRO_MEMBERS':'TRO_PONTA_ID'})
df_pv_['EVENT_TIME'] = pd.to_datetime(df_pv_['EVENT_TIME']).dt.strftime('%Y-%m')

df_pv = df_pv_.groupby(['TRO_PONTA_ID','EVENT_TIME']).agg(PV=('COUNT_VIEW_PRODUCT','sum'))\
        .unstack(level=1).fillna(0).reset_index()

df_pv.columns = [' '.join(col).strip().upper() for col in df_pv.columns.values]

In [3]:
df_re_g_u = pd.read_csv('./feature/rating_2022-07-01.csv')

In [4]:
df_re_g_u = df_re_g_u.rename(columns = {'TRO_MEMBERS':'TRO_PONTA_ID'})

In [5]:
# HIDE CODE

## USING ORDER
import glob

df_tto = []
for end_date in pd.date_range('2022-03-01', '2022-07-01', freq='M'):
    start_date = end_date.replace(day=1)
    end_date_save = start_date + relativedelta(months=1)
    
    start_date_str = start_date.strftime('%d%b%y')
    end_date_str = end_date_save.strftime('%d%b%y')
    
    print(start_date_str, end_date_str)
    
#     pf = '/home/server/gli-data-science/data/trans_oshop_test/{}_{}.csv'.format(start_date_str, end_date_str)
#     df_tto.append(pd.read_csv(pf, sep='\t', dtype='object'))
    pf = '/home/server/gli-data-science/data/trans_oshop/{}_{}.parquet'.format(start_date_str, end_date_str)
    df_tto.append(pd.read_parquet(pf))
    
    
df_tto = pd.concat(df_tto)

# df_tto = df_tto[df_tto['TRO_PONTA_ID'] != 'N']
df_tto['TRO_DATE'] = pd.to_datetime(df_tto['TRO_DATE'])
df_tto['TRO_NET'] = df_tto['TRO_NET'].astype('float', errors='ignore').astype('int', errors='ignore')


df_tto['TRO_QTY'] = df_tto['TRO_QTY'].astype(int)

01Mar22 01Apr22
01Apr22 01May22
01May22 01Jun22
01Jun22 01Jul22


In [6]:
## USE THIS FOR GET PV DATA

df_mem_push = df_tto[['TRO_PONTA_ID']].drop_duplicates()
from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

engine = create_engine(engine_stmt)

df = df_mem_push.astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()





In [7]:
df_tto['TRO_VOUCHER_USAGE'] = df_tto['TRO_VOUCHER_USAGE']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')
df_tto['TRO_DH_SPESIAL'] = df_tto['TRO_DH_SPESIAL']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')


df_tto_sel = df_tto[(df_tto['TRO_DATE'] >= '2022-03-01') \
            & (df_tto['TRO_DATE'] <= '2022-06-30')]

In [8]:
df_sla = pd.read_csv('./feature/sla_30Jun22.csv')
df_sla['ATT_SEND_DATE_TOSTORE'] = pd.to_datetime(df_sla['ATT_SEND_DATE_TOSTORE'])
df_sla['SLA'] = np.where(df_sla['SLA_NORM']< 0, 20, df_sla['SLA_NORM'])

df_tto_sel = pd.merge(df_tto_sel, df_sla, left_on='TRO_NO_ORDER', right_on='ATT_ORDER_ID', how='left')

col_drop = [
    'ATT_ORDER_ID',
    'ATT_ORDER_DATE',
    'ATT_DELIVERY_DATE',
    'ATT_SEND_DATE_TOSTORE',
    'SLA_NORM'        
]
df_tto_sel = df_tto_sel.drop(col_drop,1)

## create feature
df_tto_sel['SLA'] = df_tto_sel['SLA'].fillna(20)
df_tto_sel['SLA_LATE'] = np.where(df_tto_sel['SLA'] > 60, 1,0)

In [9]:
q = '''
SELECT 
	tc.TRO_PONTA_ID, 
	(TO_DATE('2022/06/08', 'yyyy/mm/dd')) - zt.tgl_registrasi AS SINCE_REGIS
FROM 
	TEMP_CHURN tc
LEFT JOIN (
	select 
		char1 no_member, 
        date2 tgl_registrasi
	from 
		report.ztampungan
	where 
		pk='MEMBERALFAGIFT2'
) zt ON zt.no_member = tc.TRO_PONTA_ID 

'''

con = ds_db.connect_alfabi()

df_regis = pd.read_sql_query(q, con)

con.close()

You are connected


In [10]:
df_regis['SINCE_REGIS'] = df_regis['SINCE_REGIS'].fillna('-1')

In [12]:
df_tto_sel['TRO_DATE_MONTH'] = (df_tto_sel['TRO_DATE'].dt.year.astype(str)
                                + '-0' + df_tto_sel['TRO_DATE'].dt.month.astype(str))

In [13]:
df_tto_sel_g = df_tto_sel.groupby(['TRO_PONTA_ID','TRO_DATE_MONTH','TRO_NO_ORDER'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'nunique',
                              'SLA':'first',
                              'SLA_LATE':'first',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'first'
                             })
df_tto_sel_g = df_tto_sel_g.groupby(['TRO_PONTA_ID','TRO_DATE_MONTH'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'sum',
                              'SLA':'sum',
                              'SLA_LATE':'sum',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'nunique'
                             })

## unstack all feature in oshop

In [14]:
df_tto_sel_g_u = df_tto_sel_g.unstack(level=1).fillna(0).reset_index()
df_tto_sel_g_u.columns = [' '.join(col).strip().upper() for col in df_tto_sel_g_u.columns.values]

## merge with rating

In [15]:
df_re_g_u['TRO_PONTA_ID'] = df_re_g_u['TRO_PONTA_ID'].astype(str)
df_tto_re = pd.merge(df_tto_sel_g_u, df_re_g_u, on='TRO_PONTA_ID', how='left').fillna(0)

## merge with product view

In [16]:
df_pv['TRO_PONTA_ID'] = df_pv['TRO_PONTA_ID'].astype(str)
df_tto_re_pv = pd.merge(df_tto_re, df_pv, on='TRO_PONTA_ID', how='left').fillna(0)

## merge with regis

In [17]:
df_tto_re_pv = pd.merge(df_tto_re_pv, df_regis, on='TRO_PONTA_ID')
df_tto_re_pv = df_tto_re_pv[df_tto_re_pv['SINCE_REGIS'].astype(int) > 0]

In [18]:
df_tto_re_pv.head()

,TRO_PONTA_ID,TRO_NET 2022-03,TRO_NET 2022-04,TRO_NET 2022-05,TRO_NET 2022-06,TRO_DH_SPESIAL 2022-03,TRO_DH_SPESIAL 2022-04,TRO_DH_SPESIAL 2022-05,TRO_DH_SPESIAL 2022-06,TRO_NO_ORDER 2022-03,TRO_NO_ORDER 2022-04,TRO_NO_ORDER 2022-05,TRO_NO_ORDER 2022-06,SLA 2022-03,SLA 2022-04,SLA 2022-05,SLA 2022-06,SLA_LATE 2022-03,SLA_LATE 2022-04,SLA_LATE 2022-05,SLA_LATE 2022-06,TRO_VOUCHER_USAGE 2022-03,TRO_VOUCHER_USAGE 2022-04,TRO_VOUCHER_USAGE 2022-05,TRO_VOUCHER_USAGE 2022-06,TRO_PAYMENT_METHOD 2022-03,TRO_PAYMENT_METHOD 2022-04,TRO_PAYMENT_METHOD 2022-05,TRO_PAYMENT_METHOD 2022-06,RATING_BELOW 2022-03,RATING_BELOW 2022-04,RATING_BELOW 2022-05,RATING_BELOW 2022-06,RATING_AVG 2022-03,RATING_AVG 2022-04,RATING_AVG 2022-05,RATING_AVG 2022-06,PV 2022-03,PV 2022-04,PV 2022-05,PV 2022-06,SINCE_REGIS
0,9990010000001200,1686956.0,4846037.0,192099.0,264662.0,114900.0,419599.0,17699.0,29100.0,19.0,23.0,4.0,6.0,227.2,313.4,44.2,82.1,0.0,0.0,0.0,0.0,0.0,30000.0,64000.0,5430000.0,3.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,17.0,13.0,0.0,5.0,1114.0
1,9990010000001771,343725.0,378647.0,552242.0,748387.0,42300.0,6700.0,84299.0,94908.0,1.0,1.0,6.0,30.0,2.5,3.0,81.5,280.9,0.0,0.0,0.0,0.0,0.0,1600000.0,244000.0,2778000.0,1.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,3.0,2.0,1049.0
2,9990010000001999,0.0,0.0,0.0,59480.0,0.0,0.0,0.0,5900.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1870.8,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2466000.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1087.0
3,9990010000003113,0.0,0.0,0.0,195671.0,0.0,0.0,0.0,54200.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,38.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,942.0
4,9990010000008534,146724.0,210603.0,94676.0,138813.0,54200.0,30400.0,6900.0,7900.0,2.0,2.0,12.0,23.0,10.7,20.4,250.2,432.1,0.0,0.0,0.0,0.0,0.0,0.0,121000.0,3427000.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,10.0,1109.0


In [19]:
# df_tto_re_pv['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# df_tto_re_pv['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-10'] - df_tto_re_pv['TRO_NET 2021-09']
# df_tto_re_pv['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-09']

# df_tto_re_pv['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-10'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']
# df_tto_re_pv['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['PV 11-10'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# df_tto_re_pv['PV 10-09'] = df_tto_re_pv['PV 2021-10'] - df_tto_re_pv['PV 2021-09']
# df_tto_re_pv['PV 11-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-09']

# df_tto_re_pv['SLA 11-10'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# df_tto_re_pv['SLA 10-09'] = df_tto_re_pv['SLA 2021-10'] - df_tto_re_pv['SLA 2021-09']
# df_tto_re_pv['SLA 11-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-09']

# df_tto_re_pv['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# df_tto_re_pv['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-09']/df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [20]:
# print(df_ins[df_ins['label'] == 1]['NET_11-10'].median())
# print(df_ins[df_ins['label'] == 1]['NET_10-09'].median())
# print(df_ins[df_ins['label'] == 1]['NET_11-09'].median())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-10'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_10-09'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-09'].mean())

## Random Forest

### not oversampling version without threshold 2/3 or 3/3

In [22]:
from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2022-03'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-04'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-05'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-06'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2022-03'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-04'] == 0) & \
(df_tto_re_pv['TRO_NET 2022-05'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-06'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2022-03'] == 0) & \
(df_tto_re_pv['TRO_NET 2022-04'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-05'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-06'] == 0))
)

condition_3_3 = ( 
          (df_tto_re_pv['TRO_NET 2022-03'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-04'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-05'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-06'] == 0) 
)

df_tto_re_pv['label'] = np.where(condition_3_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_PONTA_ID',
'TRO_NET 2022-06', 
'TRO_DH_SPESIAL 2022-06',
'TRO_NO_ORDER 2022-06',
'SLA 2022-06',
'SLA_LATE 2022-06',
'RATING_BELOW 2022-06',
'RATING_AVG 2022-06',
'PV 2022-06',
'TRO_VOUCHER_USAGE 2022-06',
'TRO_PAYMENT_METHOD 2022-06',
'label'
], 1
)
y = df_tto_re_pv[['label']]


# X['NET-1'] = X.iloc[:,2] - X.iloc[:,1]
# X['NET-2'] = X.iloc[:,1] - X.iloc[:,0]


# X['ORDER-1'] = X.iloc[:,8] - X.iloc[:,7]
# X['ORDER-2'] = X.iloc[:,7] - X.iloc[:,6]


# X['SLA-1'] = X.iloc[:,11] - X.iloc[:,10]
# X['SLA-2'] = X.iloc[:,10] - X.iloc[:,9]


# X['PV-1'] = X.iloc[:,29] - X.iloc[:,28]
# X['PV-2'] = X.iloc[:,28] - X.iloc[:,27]


# skf = StratifiedKFold(n_splits=5)
# for train_index, test_index in skf.split(X, y):
#     print("="*50)
#     X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
#     y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]


    
#     ## applying standard scaler
#     scaler = StandardScaler()
#     scaler.fit(X_train)
#     X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
#     X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_train.columns)
    

#     #clf_ridge = RandomForestClassifier(random_state=42) 
#     clf_ridge = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
#     clf_ridge.fit(X_train, y_train) 
#     pred = clf_ridge.predict(X_test)
#     print(classification_report(y_test, pred))
#     fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
#     ConfusionMatrixDisplay.from_predictions(y_test, pred, 
#                                             cmap='Blues', display_labels= ['not_churn', 'churn'], 
#                                             ax = ax, values_format = 'd')


# applying standard scaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
dump(scaler, './model/scaler_70_jun22_3_3.joblib')


clf_all_mlp = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
clf_all_mlp.fit(X, y) 
dump(clf_all_mlp, './model/mlp_70_jun22_3_3.joblib')




['./model/mlp_70_jun22_3_3.joblib']

In [23]:
# a = pd.DataFrame()

# a['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-11']
# a['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# a['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-10']

# a['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# a['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['PV 11-10'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-11']
# a['PV 10-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# a['PV 11-09'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-10']

# a['SLA 11-10'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-11']
# a['SLA 10-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# a['SLA 11-09'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-10']

# a['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-12']/df_tto_re_pv['TRO_NO_ORDER 2021-12']
# a['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [24]:
X_12 = df_tto_re_pv.drop(
[
'TRO_PONTA_ID',
'TRO_NET 2022-03', 
'TRO_DH_SPESIAL 2022-03',
'TRO_NO_ORDER 2022-03',
'SLA 2022-03',
'SLA_LATE 2022-03',
'RATING_BELOW 2022-03',
'RATING_AVG 2022-03',
'PV 2022-03',
'TRO_VOUCHER_USAGE 2022-03',
'TRO_PAYMENT_METHOD 2022-03',
'label'
], 1
)
X_12['SINCE_REGIS'] = X_12['SINCE_REGIS']+30

# X_12 = pd.concat([X_12.iloc[:,0:-16],a],1)
# X_12 = X_12.fillna(0)
# y_12 = df_tto_re_pv['label']

In [25]:
X_12.shape

(1324686, 31)

In [26]:
# X_12.head()

In [38]:


scaler_all = load('./model/scaler_70_jun22_3_3.joblib')
clf_all = load('./model/mlp_70_jun22_3_3.joblib')


X_12_scale = pd.DataFrame(scaler_all.transform(X_12), index=X_12.index, columns=X_12.columns)
pred = (clf_all.predict_proba(X_12_scale)[:,1] >= 0.48).astype(int)


df_mem_12 = pd.concat([df_tto_re_pv[['TRO_PONTA_ID']].reset_index(drop=True)
                       , pd.DataFrame(pred, columns=['PRED'])], 1)

In [39]:
print("PREDICTION RESULT {}".format(df_mem_12[df_mem_12['PRED'] == 1].shape))

PREDICTION RESULT (6595, 2)


In [40]:
df_trader = pd.read_excel('/home/server/gli-data-science/ferina/Sales Trader Ytd Aug-ExEmployee.xlsx'
                          , dtype='object', names=['member','stat'])

In [41]:
df_mem_12 = df_mem_12[~df_mem_12['TRO_PONTA_ID'].isin(df_trader['member'])]

In [42]:
print("AFTER REMOVE TRADER {}".format(df_mem_12[df_mem_12['PRED'] == 1].shape))

AFTER REMOVE TRADER (6503, 2)


In [43]:
from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

engine = create_engine(engine_stmt)

df = df_mem_12[df_mem_12['PRED'] == 1].astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn_target', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()

In [44]:
q = '''
SELECT DISTINCT(tct.TRO_PONTA_ID)
FROM TEMP_CHURN_TARGET tct
WHERE 
tct.TRO_PONTA_ID NOT IN (
	SELECT 
		gua.GUA_ACCOUNT_CARD_ADJ
	FROM GLI_UNINSTALL_ALFAGIFT gua 
	WHERE 
		gua.GUA_STATUS = 'Y'
		AND trunc(GUA_UNINSTALL_DATE) BETWEEN '01-jul-21' AND '31-jul-22'
)


'''

con = ds_db.connect_alfabi()
df_churn_sel = pd.read_sql_query(q, con)
con.close()

You are connected


In [45]:
df_churn_sel['PERIOD'] = '2022-07-01'
df_churn_sel['PERIOD'] = pd.to_datetime(df_churn_sel['PERIOD'])
df_churn_sel = df_churn_sel[['PERIOD','TRO_PONTA_ID']].rename(columns={'TRO_PONTA_ID':'MEMBER'})
df_churn_all = df_churn_sel.copy()

In [46]:
df_churn_all.shape

(6022, 2)

In [47]:


from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )
    
engine = create_engine(engine_stmt)

q = '''

DELETE FROM ALFAGIFT_CHURN
WHERE PERIOD = '1-jul-22'

'''

con = engine.connect()
con.execute(q)

df = df_churn_all.copy()
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('ALFAGIFT_CHURN', engine, index=False, if_exists="append", dtype=dtyp)



engine.dispose()

In [48]:
df_churn_all.head(20)

,PERIOD,MEMBER
0,2022-07-01,9990013618076900
1,2022-07-01,9990013774302925
2,2022-07-01,9990013613943410
3,2022-07-01,9990013774761116
4,2022-07-01,9990013705790770
5,2022-07-01,9990013744518440
6,2022-07-01,9990013714352700
7,2022-07-01,9990013599177548
8,2022-07-01,9990013747998638
9,2022-07-01,9990013487792650


In [49]:
df_churn_view = pd.merge(df_churn_sel,df_tto_re_pv,left_on='MEMBER'
                         ,right_on='TRO_PONTA_ID', how='left')

In [50]:
df_churn_view.head(20)

,PERIOD,MEMBER,TRO_PONTA_ID,TRO_NET 2022-03,TRO_NET 2022-04,TRO_NET 2022-05,TRO_NET 2022-06,TRO_DH_SPESIAL 2022-03,TRO_DH_SPESIAL 2022-04,TRO_DH_SPESIAL 2022-05,TRO_DH_SPESIAL 2022-06,TRO_NO_ORDER 2022-03,TRO_NO_ORDER 2022-04,TRO_NO_ORDER 2022-05,TRO_NO_ORDER 2022-06,SLA 2022-03,SLA 2022-04,SLA 2022-05,SLA 2022-06,SLA_LATE 2022-03,SLA_LATE 2022-04,SLA_LATE 2022-05,SLA_LATE 2022-06,TRO_VOUCHER_USAGE 2022-03,TRO_VOUCHER_USAGE 2022-04,TRO_VOUCHER_USAGE 2022-05,TRO_VOUCHER_USAGE 2022-06,TRO_PAYMENT_METHOD 2022-03,TRO_PAYMENT_METHOD 2022-04,TRO_PAYMENT_METHOD 2022-05,TRO_PAYMENT_METHOD 2022-06,RATING_BELOW 2022-03,RATING_BELOW 2022-04,RATING_BELOW 2022-05,RATING_BELOW 2022-06,RATING_AVG 2022-03,RATING_AVG 2022-04,RATING_AVG 2022-05,RATING_AVG 2022-06,PV 2022-03,PV 2022-04,PV 2022-05,PV 2022-06,SINCE_REGIS,label
0,2022-07-01,9990013618076900,9990013618076900,78725.0,36035.0,36035.0,36035.0,9200.0,200.0,200.0,200.0,2.0,1.0,1.0,1.0,25.2,20.0,47.4,8.5,0.0,0.0,0.0,0.0,12000.0,0.0,0.0,10000.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,122.0,0
1,2022-07-01,9990013774302925,9990013774302925,0.0,129368.0,116936.0,36968.0,0.0,1000.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,13.1,7.4,10.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,43.0,0
2,2022-07-01,9990013613943410,9990013613943410,377181.0,43872.0,56304.0,87113.0,70400.0,8000.0,14900.0,11900.0,1.0,1.0,1.0,1.0,21.6,3.4,7.5,27.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,16.0,16.0,9.0,124.0,0
3,2022-07-01,9990013774761116,9990013774761116,0.0,84050.0,593831.0,66680.0,0.0,5000.0,26200.0,6300.0,0.0,1.0,6.0,1.0,0.0,4.7,70.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0,4.0,42.0,0
4,2022-07-01,9990013705790770,9990013705790770,2636.0,18198.0,13500.0,2792.0,0.0,3200.0,0.0,200.0,1.0,1.0,1.0,1.0,2.0,10.1,1.2,1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.0,0
5,2022-07-01,9990013744518440,9990013744518440,0.0,54142.0,36936.0,164951.0,0.0,400.0,0.0,22300.0,0.0,2.0,1.0,1.0,0.0,32.0,39.8,22.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,9.0,4.0,4.0,69.0,0
6,2022-07-01,9990013714352700,9990013714352700,0.0,101080.0,92972.0,86306.0,0.0,12200.0,200.0,32200.0,0.0,1.0,1.0,1.0,0.0,20.0,20.0,8.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.0,0
7,2022-07-01,9990013599177548,9990013599177548,270982.0,27836.0,88436.0,96231.0,21500.0,0.0,6700.0,9400.0,7.0,2.0,1.0,1.0,125.2,56.9,30.9,50.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,35.0,6.0,10.0,1.0,130.0,0
8,2022-07-01,9990013747998638,9990013747998638,0.0,78194.0,10900.0,10630.0,0.0,16000.0,1200.0,200.0,0.0,2.0,1.0,1.0,0.0,40.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,66.0,0
9,2022-07-01,9990013487792650,9990013487792650,0.0,1387629.0,5755242.0,103962.0,0.0,53200.0,287200.0,600.0,0.0,13.0,56.0,3.0,0.0,87.9,438.8,28.9,0.0,0.0,0.0,0.0,0.0,2400000.0,70000.0,1800000.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,29.0,170.0,38.0,176.0,0


In [51]:
from multiprocessing import Pool
MEMBER_BLAST = ['9990010153254191','9990010153254191','9990010153254191']
TGL_F = ['2022-05-01','2022-05-04','2022-05-28']
PHONE = ['081242240791','081242240791','081242240791']

df_ret = pd.DataFrame({'MEMBER_BLAST':MEMBER_BLAST,'TGL_F':TGL_F,'PHONE':PHONE})

group_col = [
    'MEMBER_BLAST',

]

gab_sel_group = df_ret.groupby(group_col)\
                .agg({'TGL_F':lambda x: list(x),
                     'PHONE':lambda x: list(x)}).reset_index()


def remark_sales(list_sales):
    #print(list_sales)
    if all(i == 'NONE' for i in list_sales):
        remark = 'NONE ALL'
    elif any(i == 'NONE' for i in list_sales):
        remark = 'NONE PARTIAL'
    elif all((i.startswith('08') or i.startswith('+628') for i in list_sales)):
        remark = 'PHONE NUMBER ALL'
    
    return remark


def add_features(df):
    
    df['REMARK_PHONE'] = df['PHONE'].apply(remark_sales)
    return df
    
    
def parallelize_dataframe(df, func, n_cores=1):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

gab_sel_group = parallelize_dataframe(gab_sel_group, add_features)
gab_sel_group

,MEMBER_BLAST,TGL_F,PHONE,REMARK_PHONE
0,9990010153254191,"[2022-05-01, 2022-05-04, 2022-05-28]","[081242240791, 081242240791, 081242240791]",PHONE NUMBER ALL


In [ ]:
MEMBER_BLAST = ['9990010153254191','9990010153254191','9990010153254191']
TGL_F = ['2022-05-01','2022-05-04','2022-05-28']
PHONE = ['08129707447','NONE','08129707447']

df_ret = pd.DataFrame({'MEMBER_BLAST':MEMBER_BLAST,'TGL_F':TGL_F,'PHONE':PHONE})

group_col = [
    'MEMBER_BLAST',

]

gab_sel_group = df_ret.groupby(group_col)\
                .agg({'TGL_F':lambda x: list(x),
                     'PHONE':lambda x: list(x)}).reset_index()

In [ ]:
gab_sel_group['REMARK_PHONE'] = gab_sel_group['PHONE'].apply(remark_sales)

In [ ]:
gab_sel_group